<table class="table table-bordered">
    <tr>
        <th style="text-align:center; width:25%"><img src='https://www.nus.edu.sg/images/default-source/base/logo.png' style="width: 250px; height: 125px; "></th>
        <th style="text-align:center;"><h1>Machine Learning in Python</h1><h2>Lab 3a - Linear Models </h2><h3></h3></th>
    </tr>
</table>

In [1]:
# Import the requried packages
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model, neighbors, tree, svm, ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score, cross_validate

### 1.Classification

We will be using the bank marketing campaign data for classification task.

In [2]:
# Loading the data
bank = pd.read_csv('bank.csv')
print(bank.head())
# Encode the categorical data into numbers
bank_cat = bank.select_dtypes(['object']).copy()
for col in bank_cat:
    print(col, ':')
    codes, uniques = pd.factorize(bank_cat[col], sort=True)
    bank[col]=codes
    print(uniques)
print(bank.head())
    
# Set the "deposit" as target/model output and the rest features as model inputs
y = bank['deposit']
X = bank.drop(['deposit'], axis=1)

# Split the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

#Sacaling the data into a smaller range (-3 to +3)
mean = X_train.mean()
std = X_train.std()

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std


   age         job  marital  education default  balance housing loan  contact  \
0   59      admin.  married  secondary      no     2343     yes   no  unknown   
1   56      admin.  married  secondary      no       45      no   no  unknown   
2   41  technician  married  secondary      no     1270     yes   no  unknown   
3   55    services  married  secondary      no     2476     yes   no  unknown   
4   54      admin.  married   tertiary      no      184      no   no  unknown   

   day month  duration  campaign  pdays  previous poutcome deposit  
0    5   may      1042         1     -1         0  unknown     yes  
1    5   may      1467         1     -1         0  unknown     yes  
2    5   may      1389         1     -1         0  unknown     yes  
3    5   may       579         1     -1         0  unknown     yes  
4    5   may       673         2     -1         0  unknown     yes  
job :
Index(['admin.', 'blue-collar', 'entrepreneur', 'housemaid', 'management',
       'retired', 

In [9]:
#Logistic Regression Model

lg_clf = linear_model.LogisticRegression() 
# Fit the model to the training set
lg_clf.fit(X_train,y_train)

LogisticRegression()

In [10]:
# print out the model coefficients and intercept
print(lg_clf.coef_)
print(lg_clf.intercept_)

[[ 0.11071648  0.06615286  0.1346478   0.15737429 -0.0517361   0.10347281
  -0.5275739  -0.24773081 -0.55150842 -0.05986311  0.13253692  1.72117011
  -0.35567351  0.33525199  0.29432498  0.19792347]]
[-0.04641553]


In [11]:
print('train_acc:', lg_clf.score(X_train, y_train), 
      '\n test_acc:', lg_clf.score(X_test, y_test))

train_acc: 0.795721805353343 
 test_acc: 0.8007165248544559


**statsmodels**

In [15]:
# Create another model lg_clf2 using statsmodel.api: the Logit method and fit function
import statsmodels.api as sm
X_train_add = sm.add_constant(X_train)
lg_clf2 = sm.Logit(y_train, X_train_add).fit()

Optimization terminated successfully.
         Current function value: 0.469782
         Iterations 7


In [16]:
# Print out the Model Summary
lg_clf2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                deposit   No. Observations:                 8929
Model:                          Logit   Df Residuals:                     8912
Method:                           MLE   Df Model:                           16
Date:                Wed, 04 Jan 2023   Pseudo R-squ.:                  0.3210
Time:                        13:31:54   Log-Likelihood:                -4194.7
converged:                       True   LL-Null:                       -6178.1
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0461      0.028     -1.623      0.105      -0.102       0.010
age            0.1109      0.031      3.615      0.000       0.051       0.171
job            0.0662      0.028      2.400      0.016       0.012       0.120
marital        0.1348      0.031      4.361      0.000       0.074       0.195
education      0.1576      0.028      5.670      0.000       0.103       0.212
default       -0.0518      0.030     -1.747      0.081      -0.110       0.006
balance        0.1035      0.029      3.629      0.000       0.048       0.159
housing       -0.5285      0.030    -17.807      0.000      -0.587      -0.470
loan          -0.2481      0.030     -8.365      0.000      -0.306      -0.190
contact       -0.5525      0.033    -16.522      0.000      -0.618      -0.487
day           -0.0599      0.027     -2.213      0.027      -0.113      -0.007
month          0.1329      0.028      4.788      0.000       0.078       0.187
duration       1.7250      0.044     38.792      0.000       1.638       1.812
campaign      -0.3565      0.039     -9.101      0.000      -0.433      -0.280
pdays          0.3364      0.046      7.312      0.000       0.246       0.427
previous       0.2950      0.037      7.955      0.000       0.222       0.368
poutcome       0.1991      0.047      4.226      0.000       0.107       0.291
==============================================================================
"""

The lower the **p-value**, the higher confidence we have on the coefficients of these features, i.e. the more strong indicators to predict `deposit`. 

### 2. Regression
We will be using the supermarket data for regression task. 

In [17]:
# Loading the data
supermarket = pd.read_csv('supermarket.csv')
print(supermarket.head())

# Encode the categorical data into numbers
supermarket_cat = supermarket.select_dtypes(['object']).copy()
for col in supermarket_cat:
    print(col, ':')
    codes, uniques = pd.factorize(supermarket_cat[col], sort=True)
    supermarket[col]=codes
    print(uniques)
print(supermarket.head())

# Set the "Sales" as target/model output and the rest features as model inputs
y = supermarket['Item_Outlet_Sales']
X = supermarket.drop(['Item_Outlet_Sales'], axis=1)

# Split the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

#Sacaling the data into a smaller range (-3 to +3)
mean = X_train.mean()
std = X_train.std()

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

   Item_Weight Item_Fat_Content  Item_Visibility              Item_Type  \
0         9.30          Low Fat         0.016047                  Dairy   
1         5.92          Regular         0.019278            Soft Drinks   
2        17.50          Low Fat         0.016760                   Meat   
3        19.20          Regular         0.000000  Fruits and Vegetables   
4         8.93          Low Fat         0.000000              Household   

   Item_MRP Outlet_Identifier  Outlet_Establishment_Year Outlet_Size  \
0  249.8092            OUT049                       1999      Medium   
1   48.2692            OUT018                       2009      Medium   
2  141.6180            OUT049                       1999      Medium   
3  182.0950            OUT010                       1998         NaN   
4   53.8614            OUT013                       1987        High   

  Outlet_Location_Type        Outlet_Type  Item_Outlet_Sales  
0               Tier 1  Supermarket Type1          37

In [18]:
#Linear Regression Model
lm_reg = linear_model.LinearRegression()
lm_reg.fit(X_train, y_train)

LinearRegression()

In [19]:
# print out the model coefficients and intercept
print(lm_reg.coef_)
print(lm_reg.intercept_)

[ 2.83934419e-01  3.01808227e+01 -8.61083480e+01 -4.01822348e+00
  8.59688057e+02  1.97666270e+02 -9.91942511e+00 -1.70374509e+02
 -1.17920844e+02  6.40338748e+02]
2073.9178811516


In [20]:
print('train_mae:', mean_absolute_error(lm_reg.predict(X_train), y_train), 
      '\n test_mae:', mean_absolute_error(lm_reg.predict(X_test), y_test))

train_mae: 835.9531497512953 
 test_mae: 813.5881859608403


In [21]:
supermarket.Item_Outlet_Sales.describe()

count    8337.000000
mean     2059.362844
std      1506.099754
min        33.290000
25%       810.944400
50%      1747.059200
75%      2998.097400
max      6478.234000
Name: Item_Outlet_Sales, dtype: float64

In [22]:
# R squared value
lm_reg.score(X_test, y_test)

0.47876024412383733

**statsmodels**

In [23]:
# Create another model lm_reg2 using statsmodel.api: the Ordinary Least Squares (OLS) method and fit function
import statsmodels.api as sm
X_train_add = sm.add_constant(X_train)
lm_reg2 = sm.OLS(y_train, X_train_add).fit()

In [24]:
# Print out the Model Summary
lm_reg2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.492
Method:                 Least Squares   F-statistic:                     646.7
Date:                Wed, 04 Jan 2023   Prob (F-statistic):               0.00
Time:                        13:32:16   Log-Likelihood:                -56063.
No. Observations:                6669   AIC:                         1.121e+05
Df Residuals:                    6658   BIC:                         1.122e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                      2073.9179     13.274    156.239      0.000    2047.897    2099.939
Item_Weight                   0.2839     13.299      0.021      0.983     -25.786      26.354
Item_Fat_Content             30.1808     13.411      2.250      0.024       3.891      56.471
Item_Visibility             -86.1083     13.638     -6.314      0.000    -112.844     -59.373
Item_Type                    -4.0182     13.420     -0.299      0.765     -30.326      22.290
Item_MRP                    859.6881     13.302     64.626      0.000     833.611     885.765
Outlet_Identifier           197.6663     27.182      7.272      0.000     144.381     250.952
Outlet_Establishment_Year    -9.9194     13.923     -0.712      0.476     -37.212      17.373
Outlet_Size                -170.3745     16.423    -10.374      0.000    -202.569    -138.180
Outlet_Location_Type       -117.9208     31.317     -3.765      0.000    -179.313     -56.529
Outlet_Type                 640.3387     22.003     29.102      0.000     597.205     683.473
==============================================================================
Omnibus:                      162.408   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              185.448
Skew:                           0.345   Prob(JB):                     5.38e-41
Kurtosis:                       3.436   Cond. No.                         4.75
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The lower the **p-value**, the higher confidence we have on the coefficients of these features, i.e. the more strong indicators to predict `deposit`. 